# Scroll down to get to the interesting tables...

# Construct list of properties of widgets

"Properties" here is one of:

+ `keys`
+ `traits()`
+ `class_own_traits()`

Common (i.e. uninteresting) properties are filtered out.

The dependency on astropy is for their Table. Replace it with pandas if you want...

In [1]:
import itertools
from ipywidgets import *
from IPython.display import display
from traitlets import TraitError

from astropy.table import Table, Column

# Function definitions

## Calculate "interesting" properties

In [2]:
def properties(widget, omit=None, source=None):
    """
    Return a list of widget properties for a widget instance, omitting
    common properties.
    
    Parameters
    ----------
    
    widget : ipywidgets.Widget instance
        The widget for which the list of preoperties is desired.
    omit : list, optional
        List of properties to omit in the return value. Default is 
        ``['layout', 'style', 'msg_throttle']``, and for `source='traits'
        is extended to add ``['keys', 'comm']``.
    source : str, one of 'keys', 'traits', 'class_own_traits', 'style_keys' optional
        Source of property list for widget. Default is ``'keys'``.
    """
    if source is None:
        source = 'keys'
    valid_sources = ('keys', 'traits', 'class_own_traits', 'style_keys')
    if source not in valid_sources:
        raise ValueError('source must be one of {}'.format(', '.join(valid_sources)))
    if omit is None:
        omit = ['layout', 'style', 'msg_throttle']
    if source == 'keys':
        props = widget.keys
    elif source == 'traits':
        props = widget.traits()
        omit.extend(['keys', 'comm'])
    elif source == 'class_own_traits':
        props = widget.class_own_traits()
    elif source == 'style_keys':
        props = widget.style.keys
    props = [k for k in props if not k.startswith('_')]    
    return [k for k in props if k not in omit]

## Create a table (cross-tab style) for which properties are available for which widgets

This is the only place astropy.table.Table is used, so delete if you want to.

In [3]:
def table_for_keys(keys, keys_info, source):
    unique_keys = set()
    for k in keys:
        unique_keys.update(keys_info[k])
    unique_keys = sorted(unique_keys)
    string_it = lambda x: 'X' if x else ''
    colnames = ['Property ({})'.format(source)] + keys

    columns = [Column(name=colnames[0], data=unique_keys)]
    for c in colnames[1:]:
        column = Column(name=c, data=[string_it(k in key_dict[c]) for k in unique_keys])
        columns.append(column)
    return Table(columns)

## List of widget objects...

In [4]:
widget_list = [
    IntSlider,
    FloatSlider,
    IntRangeSlider,
    FloatRangeSlider,
    IntProgress,
    FloatProgress,
    BoundedIntText,
    BoundedFloatText,
    IntText,
    FloatText,
    ToggleButton,
    Checkbox,
    Valid,
    Dropdown,
    RadioButtons,
    Select,
    SelectionSlider,
    SelectionRangeSlider,
    ToggleButtons,
    SelectMultiple,
    Text,
    Textarea,
    Label,
    HTML,
    HTMLMath,
    Image,
    Button,
    Play,
    DatePicker,
    ColorPicker,
    Box,
    HBox,
    VBox,
    Accordion,
    Tab
]

## ...and their names

In [5]:
names = [wd.__name__ for wd in widget_list]

## Figure out the properties for each widget

The `try`/`except` below is to catch a couple of classes that *require* that `options` be passed on intialization.

In [6]:
property_source = 'keys'
all_keys = []
for widget_class in widget_list:
    try:
        keys = properties(widget_class(), source=property_source)
    except TraitError as e:
        keys = properties(widget_class(options=(2,10)), source=property_source)
    finally:
        all_keys.append(keys)

Probably should have used a dict from the beginning...

In [7]:
key_dict = {k: v for k, v in zip(names, all_keys)}

## Define a few groups of widgets by widget interface type

This makes for nicer (i.e. more compact and readable) tables later on.

In [8]:
sliders = [k for k in key_dict.keys() if 'Slider' in k]
buttons = [k for k in key_dict.keys() if 'Button' in k]
containers = ['Box', 'VBox', 'HBox', 'Accordion', 'Tab']
texts = [k for k in names if 'text' in k or 'Text' in k] + [k for k in names if 'HTML' in k] + ['Label']
progress = [k for k in names if 'Progress' in k]
selects = ['Dropdown', 'Select', 'SelectMultiple']
all_so_far = sliders + buttons + texts + containers + progress + selects
others = [k for k in names if k not in all_so_far]

slider_keys = set()


# Tables of keys (synced properties)

## Sliders

In [9]:
table_for_keys(sliders, key_dict, source=property_source)

Property (keys),IntSlider,FloatSlider,IntRangeSlider,FloatRangeSlider,SelectionSlider,SelectionRangeSlider
str17,str1,str1,str1,str1,str1,str1
continuous_update,X,X,X,X,X,X
description,X,X,X,X,X,X
disabled,X,X,X,X,X,X
index,,,,,X,X
max,X,X,X,X,,
min,X,X,X,X,,
orientation,X,X,X,X,X,X
readout,X,X,X,X,X,X
readout_format,X,X,X,X,,


## Buttons

In [10]:
table_for_keys(buttons, key_dict, source=property_source)

Property (keys),ToggleButton,RadioButtons,ToggleButtons,Button
str12,str1,str1,str1,str1
button_style,X,,X,X
description,X,X,X,X
disabled,X,X,X,X
icon,X,,,X
icons,,,X,
index,,X,X,
tooltip,X,,,X
tooltips,,,X,
value,X,,,


## Containers

In [11]:
table_for_keys(containers, key_dict, source=property_source)

Property (keys),Box,VBox,HBox,Accordion,Tab
str14,str1,str1,str1,str1,str1
box_style,X,X,X,X,X
children,X,X,X,X,X
selected_index,,,,X,X


## Text

In [12]:
table_for_keys(texts, key_dict, source=property_source)

Property (keys),BoundedIntText,BoundedFloatText,IntText,FloatText,Text,Textarea,HTML,HTMLMath,Label
str11,str1,str1,str1,str1,str1,str1,str1,str1,str1
description,X,X,X,X,X,X,X,X,X
disabled,X,X,X,X,X,X,,,
max,X,X,,,,,,,
min,X,X,,,,,,,
placeholder,,,,,X,X,X,X,X
rows,,,,,,X,,,
value,X,X,X,X,X,X,X,X,X


## Progress bars

In [13]:
table_for_keys(progress, key_dict, source=property_source)

Property (keys),IntProgress,FloatProgress
str11,str1,str1
bar_style,X,X
description,X,X
max,X,X
min,X,X
orientation,X,X
value,X,X


# Select widgets

In [14]:
table_for_keys(selects, key_dict, source=property_source)

Property (keys),Dropdown,Select,SelectMultiple
str11,str1,str1,str1
description,X,X,X
disabled,X,X,X
index,X,X,X
rows,,X,X


## Everything else

In [15]:
table_for_keys(others, key_dict, source=property_source)

Property (keys),Checkbox,Valid,Image,Play,DatePicker,ColorPicker
str11,str1,str1,str1,str1,str1,str1
concise,,,,,,X
description,X,X,,X,X,X
disabled,X,X,,X,X,X
format,,,X,,,
height,,,X,,,
indent,X,,,,,
interval,,,,X,,
max,,,,X,,
min,,,,X,,


In [16]:
property_source = 'style_keys'
style_keys = []
for widget_class in widget_list:
    try:
        keys = properties(widget_class(), source=property_source)
    except TraitError as e:
        keys = properties(widget_class(options=(2,10)), source=property_source)
    except AttributeError:
        keys=''
    finally:
        style_keys.append(keys)

In [17]:
for w, s in zip(names, style_keys):
    print('{} has style keys: {}'.format(w, ', '.join(s)))

IntSlider has style keys: description_width, handle_color
FloatSlider has style keys: description_width, handle_color
IntRangeSlider has style keys: description_width, handle_color
FloatRangeSlider has style keys: description_width, handle_color
IntProgress has style keys: bar_color, description_width
FloatProgress has style keys: bar_color, description_width
BoundedIntText has style keys: description_width
BoundedFloatText has style keys: description_width
IntText has style keys: description_width
FloatText has style keys: description_width
ToggleButton has style keys: description_width
Checkbox has style keys: description_width
Valid has style keys: description_width
Dropdown has style keys: description_width
RadioButtons has style keys: description_width
Select has style keys: description_width
SelectionSlider has style keys: description_width
SelectionRangeSlider has style keys: description_width
ToggleButtons has style keys: description_width
SelectMultiple has style keys: descrip